<a href="https://colab.research.google.com/github/JosegSuarezb/Aprendizaje-supervisado/blob/main/Reto_2_Regresi%C3%B3n_y_regularizaci%C3%B3n_Jose_Gregorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reto 2

1. Carga el archivo `auto-mpg.data`. Ponle nombre a las columnas de acuerdo a lo indicado en el archivo `auto-mpg.names`.

2. Configura como índice la variable `car_name`.

3. Codifica un modelo de regresión Ridge que prediga la variable `mpg` usando todas las demás variables disponibles como predictoras. Para esto realiza una partición de los datos en subconjuntos de entrenamiento y prueba en una proporción 70/30, usando random_state=1.

4. Codifica un modelo de regresión Lasso que prediga la variable `mpg` usando todas las demás variables disponibles como predictoras. Para esto realiza una partición de los datos en subconjuntos de entrenamiento y prueba en una proporción 70/30, usando `random_state=1`.

5. Recuerda identificar si es necesario utilizar alguna estrategia de preprocesamiento.

6. Reporta el mejor valor de  λ  encontrado para los puntos 3 y 4, el RMSE de entrenamiento y el de prueba con este valor.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#cargamos los datos
carros= pd.read_csv('/content/drive/MyDrive/Aprendizaje supervisado/auto-mpg (1).data-original',header=None, sep='\s+')
carros.dropna()
carros.head()

,0,1,2,3,4,5,6,7,8
0,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
1,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
2,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
3,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
4,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino


#NOMBRES A LAS COLUMNAS

In [ ]:
carros.columns=['mpg','cylinders','displacement','horsepower','weight','acceleration','model_year','origin','car_name']
carros.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
1,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
2,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
3,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
4,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino


In [ ]:
carros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     406 non-null    float64
 2   displacement  406 non-null    float64
 3   horsepower    400 non-null    float64
 4   weight        406 non-null    float64
 5   acceleration  406 non-null    float64
 6   model_year    406 non-null    float64
 7   origin        406 non-null    float64
 8   car_name      406 non-null    object 
dtypes: float64(8), object(1)
memory usage: 28.7+ KB


#se coloca la variable car_name como index

In [ ]:
carros.set_index('car_name', inplace=True)
carros.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
car_name,,,,,,,,
chevrolet chevelle malibu,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
buick skylark 320,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0
plymouth satellite,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0
amc rebel sst,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0
ford torino,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0


In [ ]:
carros.dropna(inplace=True)
print(carros.isnull().sum())

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model_year      0
origin          0
dtype: int64


In [ ]:
X=carros.drop('mpg', axis=1)

In [ ]:
y=carros['mpg']

In [ ]:
y.shape

(392,)

In [ ]:
X.shape

(392, 7)

#Modelos

In [ ]:
#Importamos librerias adicionales
from sklearn.linear_model import Ridge, Lasso #para el modelo de regresión Ridge
from sklearn.metrics import mean_squared_error #para calcular el RMSE
from sklearn.model_selection import train_test_split #para dividir el conjunto de entrenamiento y prueba
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=6)
#transformacion de los datos
# Transformación de datos
ct = ColumnTransformer([
    ('standard', StandardScaler(), ['origin', 'acceleration', 'cylinders']),
    ('maxabs', MaxAbsScaler(), ['model_year', 'weight', 'displacement', 'horsepower'])
], remainder='passthrough')
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)


In [ ]:
# Definir modelo y búsqueda de hiperparámetros con GridSearchCV
param_grid = {'alpha': np.logspace(-4, 4, 50)}

In [ ]:
#Ridge
ridge = GridSearchCV(Ridge(), param_grid, scoring='neg_mean_squared_error', cv=5)
ridge.fit(X_train, y_train)
best_ridge = ridge.best_estimator_

# Predicciones y evaluación de modelos
y_train_pred_ridge = best_ridge.predict(X_train)
y_test_pred_ridge = best_ridge.predict(X_test)
rmse_train_ridge = np.sqrt(mean_squared_error(y_train, y_train_pred_ridge))
rmse_test_ridge = np.sqrt(mean_squared_error(y_test, y_test_pred_ridge))

# Reporte de resultados
print("Ridge Regression:")
print(f"Mejor valor de α: {best_ridge.alpha}")
print(f"RMSE de entrenamiento: {rmse_train_ridge}")
print(f"RMSE de prueba: {rmse_test_ridge}")


Ridge Regression:
Mejor valor de α: 0.0029470517025518097
RMSE de entrenamiento: 3.3417730090973405
RMSE de prueba: 3.239889937585057


In [ ]:
#Lasso
lasso = GridSearchCV(Lasso(), param_grid, scoring='neg_mean_squared_error', cv=5)
lasso.fit(X_train, y_train)
best_lasso = lasso.best_estimator_

y_train_pred_lasso = best_lasso.predict(X_train)
y_test_pred_lasso = best_lasso.predict(X_test)
rmse_train_lasso = np.sqrt(mean_squared_error(y_train, y_train_pred_lasso))
rmse_test_lasso = np.sqrt(mean_squared_error(y_test, y_test_pred_lasso))

print("\nLasso Regression:")
print(f"Mejor valor de α: {best_lasso.alpha}")
print(f"RMSE de entrenamiento: {rmse_train_lasso}")
print(f"RMSE de prueba: {rmse_test_lasso}")



Lasso Regression:
Mejor valor de α: 0.0001
RMSE de entrenamiento: 3.341715585519332
RMSE de prueba: 3.2418103205380118
